In [ ]:
# Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import VotingClassifier, StackingClassifier

# Target encoding/decoding
from sklearn.base import BaseEstimator, TransformerMixin

# Metrics
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, auc, roc_curve, log_loss

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, plot_importance
from catboost import CatBoostClassifier

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Math and DataFrame
import pandas as pd
import numpy as np

# Warnings ignore
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
original = pd.read_csv('/content/steel_plates_faults_original_dataset.csv')
train.head()

In [ ]:
train.shape

In [ ]:
train = pd.concat([train, original], axis=0).drop_duplicates()
train.head()

In [ ]:
train.shape

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)
train.head()

In [ ]:
target_variables = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps',  'Other_Faults']

In [ ]:
# If the sum of the axis is greater than 1 because in the same row more than 1 columns value is 1, we can sure that is not a multi class problem, it is multi label problem.
no_faults = train[train[target_variables].sum(axis=1) == 0][target_variables]
print(f'\nNumber of No Faults: {no_faults.shape[0]}\n')
no_faults

In [ ]:
no_faults.shape[0] / train.shape[0] * 100

In [ ]:
more_than_one_faults = train[train[target_variables].sum(axis=1) > 1][target_variables]
print(f'\nNumber of More than one faults: {more_than_one_faults.shape[0]}\n')
more_than_one_faults

In [ ]:
train = train[train[target_variables].sum(axis=1) <= 1]
train

In [ ]:
# Create a numpy array from our given targets.
targets_arr = train[target_variables].values.copy()
targets_arr

In [ ]:
no_faults_arr = 1 - targets_arr.sum(axis=1)[:, np.newaxis]
print(f'\nNumber of the No Faults: {no_faults_arr.sum()}\n')
no_faults_arr

In [ ]:
targets_concatenated = np.concatenate([targets_arr, no_faults_arr], axis=1)
targets_concatenated

In [ ]:
X = train.drop(target_variables, axis=1)
target = train[target_variables]
X.head()

In [ ]:
scaler = StandardScaler()
X_sc = scaler.fit_transform(X)
test_sc = scaler.transform(test)
X_sc